## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension
%matplotlib inline
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tasiilaq,GL,65.6145,-37.6368,36.86,97,100,12.86,moderate rain
1,1,Trapani,IT,37.8333,12.6667,59.27,56,100,5.73,overcast clouds
2,2,Vostok,RU,46.4856,135.8833,45.91,42,98,10.36,overcast clouds
3,3,Butaritari,KI,3.0707,172.7902,81.75,73,46,11.07,scattered clouds
4,4,Barrow,US,71.2906,-156.7887,6.82,78,0,26.46,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Butaritari,KI,3.0707,172.7902,81.75,73,46,11.07,scattered clouds
10,10,Poum,NC,-20.2333,164.0167,78.19,70,20,13.09,few clouds
14,14,Kapaa,US,22.0752,-159.3190,76.48,77,100,5.99,overcast clouds
15,15,Grand Gaube,MU,-20.0064,57.6608,77.40,87,68,7.00,light rain
16,16,Vaini,TO,-21.2000,-175.2000,84.36,70,20,8.05,few clouds
18,18,Akyab,MM,20.1500,92.9000,85.66,66,54,6.53,broken clouds
19,19,Falam,MM,22.9167,93.6833,78.01,35,45,2.51,scattered clouds
21,21,Rikitea,PF,-23.1203,-134.9692,77.79,72,3,14.47,clear sky
33,33,Quatre Cocos,MU,-20.2078,57.7625,77.11,79,92,21.97,light rain
38,38,Hilo,US,19.7297,-155.0900,80.28,89,100,5.01,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                201
City                   201
Country                201
Lat                    201
Lng                    201
Max Temp               201
Humidity               201
Cloudiness             201
Wind Speed             201
Current Description    201
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#not applicable. Would have used dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.75,scattered clouds,3.0707,172.7902,
10,Poum,NC,78.19,few clouds,-20.2333,164.0167,
14,Kapaa,US,76.48,overcast clouds,22.0752,-159.3190,
15,Grand Gaube,MU,77.40,light rain,-20.0064,57.6608,
16,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,
18,Akyab,MM,85.66,broken clouds,20.1500,92.9000,
19,Falam,MM,78.01,scattered clouds,22.9167,93.6833,
21,Rikitea,PF,77.79,clear sky,-23.1203,-134.9692,
33,Quatre Cocos,MU,77.11,light rain,-20.2078,57.7625,
38,Hilo,US,80.28,light rain,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping.")   
        

Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.75,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
10,Poum,NC,78.19,few clouds,-20.2333,164.0167,
14,Kapaa,US,76.48,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,Grand Gaube,MU,77.40,light rain,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
16,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Akyab,MM,85.66,broken clouds,20.1500,92.9000,Hotel Memory
19,Falam,MM,78.01,scattered clouds,22.9167,93.6833,Falang Government Guesthouse
21,Rikitea,PF,77.79,clear sky,-23.1203,-134.9692,People ThankYou
33,Quatre Cocos,MU,77.11,light rain,-20.2078,57.7625,Sunrise Attitude Hotel
38,Hilo,US,80.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", float("NaN"), inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.75,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
14,Kapaa,US,76.48,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,Grand Gaube,MU,77.40,light rain,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
16,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Akyab,MM,85.66,broken clouds,20.1500,92.9000,Hotel Memory
19,Falam,MM,78.01,scattered clouds,22.9167,93.6833,Falang Government Guesthouse
21,Rikitea,PF,77.79,clear sky,-23.1203,-134.9692,People ThankYou
33,Quatre Cocos,MU,77.11,light rain,-20.2078,57.7625,Sunrise Attitude Hotel
38,Hilo,US,80.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
44,Atuona,PF,79.23,light rain,-9.8000,-139.0333,Villa Enata


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))